## 42.1 데코레이터 만들기 
- 데코레이터란? : 함수(메서드)를 장식하는 도구
- ex) @staticmethod, @classmetod, @abstractmethod 등 
- 함수를 수정하지 않은 상태에서 추가 기능을 구현할 때 사용함  

**일반 함수**

In [1]:
def hello():
    print('hello 함수 시작')
    print('hello')
    print('hello 함수 끝')
    
def world():
    print('world 함수 시작')
    print('world')    
    print('world 함수 끝') 

hello()
world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


In [3]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(): # 호출할 함수를 감싸는 함수 -> 왜 감싸지? : 해당 함수의 시작과 끝을 출력할 수 있기 때문에 
        print(func.__name__, '함수 시작')   #.__name__ : 함수 이름 출력 
        func()
        print(func.__name__, '함수 끝')
    return wrapper

def hello():
    print('hello')
    
def world():
    print('world')    

trace_hello = trace(hello)
trace_hello()
trace_world = trace(world)
trace_world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


In [4]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(): # 호출할 함수를 감싸는 함수 -> 왜 감싸지? : 해당 함수의 시작과 끝을 출력할 수 있기 때문에 
        print(func.__name__, '함수 시작')   #.__name__ : 함수 이름 출력 
        func()
        print(func.__name__, '함수 끝')
    return wrapper
@trace
def hello():
    print('hello')
@trace    
def world():
    print('world')    

hello()
world()

hello 함수 시작
hello
hello 함수 끝
world 함수 시작
world
world 함수 끝


- 함수를 감싸는 형태로 구성
- 기존 함수를 수정하지 않으면서 추가 기능을 구현할 때 사용
## 42.2 매개변수와 반환값을 처리하는 데코레이터 만들기 

In [7]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(a, b): # 호출할 함수의 매개변수와 똑같이 지정
        r = func(a, b)
        print('{0}(a={1}, b={2}) -> {3}'.format(func.__name__, a, b, r))  # 매개변수와 반환값 출력
        return r # add함수의 return값을 반환하지 않으면, add함수를 호출해도 return값을 반환받지 못한다. 
    return wrapper

@trace
def add(a, b):
    return a+b

print(add(10, 20))

add(a=10, b=20) -> 30
30


**가변 인수 함수 데코레이터**
- 매개변수의 갯수가 정해지지 않은 함수의 데코레이터
- *args, ** kwargs
    - 위치 인수, 키워드 인수
    - list, dictionary

In [10]:
def trace(func):   # 함수를 매개변수로 받음
    def wrapper(*args, **kwargs): # 위치 인수와 키워드 인수를 모두 받을 수 있도록 한다. 
        r = func(*args, **kwargs)  # unpacking
        print('{0}(args={1}, kwargs={2}) -> {3}'.format(func.__name__, args, kwargs, r))  # 매개변수와 반환값 출력
        return r # add함수의 return값을 반환하지 않으면, add함수를 호출해도 return값을 반환받지 못한다. 
    return wrapper

@trace
def get_max(*args): # 위치 인수를 사용
    return max(args)

@trace
def get_min(**kwargs):  # 키워드 인수를 사용
    return min(kwargs.values())

print(get_max(10, 20))
print(get_min(x=10, y=20, z=30))

get_max(args=(10, 20), kwargs={}) -> 20
20
get_min(args=(), kwargs={'x': 10, 'y': 20, 'z': 30}) -> 10
10
